In [1]:
import pandas as pd
from influxdb import InfluxDBClient

import json
import re
from datetime import datetime
import requests

In [2]:
#optional, jupyter notebook dislay options to see more data
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None
pd.options.display.precision = 15

In [3]:
coin_auth = json.load(open('coin_credentials'))
coin_client = InfluxDBClient(
    host='testresults.qt.io', 
    username=coin_auth['username'],
    password=coin_auth['password'],
    port=8086,
    database='coin')

In [4]:
#q = "select * from flaky_test where branch = 'dev' and time > now() - 2d"
#alternative queries:
q = "select * from crashed_test where branch = 'dev' and time > now() - 2d"
df = pd.DataFrame(coin_client.query(q).get_points())
#since we do not have stats about MacOS we get won't analyse those tests
df.drop(df[df.target_os == 'MacOS'].index, inplace=True)
#some pre-processing to group the tests by : testcase, testfunction and platform on wich it was run:
df['platform'] = df['target_os_version'] + df['target_arch']
df['full_test_params'] = df['testCase'] + '+' + df['testFunction'] +'+' + df['platform']
df.head()

,time,branch,computeHost,cpu_count,cpu_count_1,cpu_seconds,duration,host_arch,host_compiler,host_os,host_os_version,id,path,platform_id,project,result,sha1,state,target_arch,target_compiler,target_os,target_os_version,taskType,testCase,testFunction,type,platform,full_test_params
0,2023-01-16T13:14:04.925466Z,dev,little-pony.on1.qt.io,2,None,1562,781,X86_64,GCC,Linux,Ubuntu_22_04,web_qt_qt5_1673856387504,qt/qtvirtualkeyboard/a2f6ca248c5dfe9e862691d4545c95fd2e23cf45/LinuxUbuntu_22_04x86_64LinuxUbuntu_22_04x86_64GCCqtci-linux-Ubuntu-22.04-x86_64-51-40e64cDocumentation_Sccache_TestDocs_UseConfigure/6f0aeb4cdee2902f1aa3fcb1272d1dbd95fc67fd/test_1681482352,ubuntu-22.04-documentation,qt/qtvirtualkeyboard,None,429e1146070d69014bbcf271b275a24abbf8a101,Done,X86_64,GCC,Linux,Ubuntu_22_04,StatusCheck,tst_inputpanel,"Error parsing result, test crashed.",Test,Ubuntu_22_04X86_64,"tst_inputpanel+Error parsing result, test crashed.+Ubuntu_22_04X86_64"
2,2023-01-16T13:14:04.925466Z,dev,whole-duck.on1.qt.io,2,None,410,205,X86_64,MSVC2019,Windows,Windows_10_22H2,web_qt_qt5_1673856387504,qt/qtmultimedia/65de259096c378230991b2cd7ee4e09f9c6466d5/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64MSVC2019qtci-windows-10_22H2-x86_64-50-a44c48Conan_DebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/db6913113ac7a5050f89b24d3afa800896b960cd/test_1681472527,windows-10_22h2-msvc2019-developer-build,qt/qtmultimedia,None,b55c2692a2e7286533c9ade98db3ce9d03a27f53,Done,X86_64,MSVC2019,Windows,Windows_10_22H2,StatusCheck,tst_qmediaplayerbackend,"Error parsing result, test crashed.",Test,Windows_10_22H2X86_64,"tst_qmediaplayerbackend+Error parsing result, test crashed.+Windows_10_22H2X86_64"
3,2023-01-16T14:40:55.532798Z,dev,loving-mongoose.on1.qt.io,2,None,764,382,X86_64,MSVC2019,Windows,Windows_10_22H2,1681492529,qt/qtapplicationmanager/8a2ff058588a1ad730d9533b5f8923583f9df8de/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64MSVC2019qtci-windows-10_22H2-x86_64-50-a44c48Conan_DebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/0362000b73d40446f5bd54fa720ffbd65c304ac0/test_1681494649,windows-10_22h2-msvc2019-developer-build,qt/qtapplicationmanager,None,4cba6c4c9e8c06d39e525d1f5b30b4f8bd76767d,Done,X86_64,MSVC2019,Windows,Windows_10_22H2,Integration,tst_main,"Error parsing result, test crashed.",Test,Windows_10_22H2X86_64,"tst_main+Error parsing result, test crashed.+Windows_10_22H2X86_64"
4,2023-01-16T15:19:31.519925Z,dev,big-grouse.on1.qt.io,8,None,9784,1223,X86_64,GCC,Linux,RHEL_9_0,1681492351,qt/qtbase/ed749d97f3e0e981898c5dfeb4320ce82c5e69fc/LinuxRHEL_9_0x86_64AndroidAndroid_ANYx86_64GCCqtci-linux-RHEL-9.0-x86_64-50-84ae08AndroidTestRun_InsignificantTests_Sccache_UseConfigure/1d07c8d16764bda2e8db5f2dcdb6fe9fbca9eb04/test_1681493860,None,qt/qtbase,None,91ab89e048557fc7fdaff4ed91809b433e6ac68d,Insignificant,X86_64,Clang,Android,Android_ANY,Integration,tst_qbytearray_large,"Error parsing result, test crashed.",Test,Android_ANYX86_64,"tst_qbytearray_large+Error parsing result, test crashed.+Android_ANYX86_64"
5,2023-01-16T15:19:31.519925Z,dev,on-ox.on1.qt.io,4,None,12588,3147,X86_64,MSVC2022,Windows,Windows_11_22H2,1681492351,qt/qtbase/ed749d97f3e0e981898c5dfeb4320ce82c5e69fc/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2022qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure/d583d33960269f8648f93e869e33b6f57b17fdc5/test_1681493777,windows-11_22h2-msvc2022,qt/qtbase,None,91ab89e048557fc7fdaff4ed91809b433e6ac68d,Failed,X86_64,MSVC2022,Windows,Windows_11_22H2,Integration,tst_qfiledialog2,"Error parsing result, test crashed.",Test,Windows_11_22H2X86_64,"tst_qfiledialog2+Error parsing result, test crashed.+Windows_11_22H2X86_64"


# connecting  given test data with vm data 

In [5]:
#just taking first random test:
path = df.iloc[0]['path']
url = "https://testresults.qt.io/logs/{}/log.txt.gz".format(path)
#url is path to log.txt.gz with test execution data, where link to telegraf metrics is added
response = requests.get(url)
response

<Response [200]>

In [6]:
#ugly processing of respone to find the start timestamp. end timestamp. and server id
#I search for:
#INFO:workitem.py:760(23870): <a href="https://inframetrics.intra.qt.io/d/nOAsINmZz/telegraf-coin-vms?orgId=1&from=1673829274000&to=1673830968000&var-server=1681269718-122700">VM metrics
result = re.findall(r"INFO:workitem.py:[0-9]+\([0-9]+\):.+VM metrics", response.text)
result
start_time = re.findall(r"from=[0-9]+", result[0])
start_timestamp = start_time[0].split('from=')[1]
end_time = re.findall(r"to=[0-9]+", result[0])
end_timestamp = end_time[0].split('to=')[1]
server_id = re.findall(r"server=[0-9]+-[0-9]+", result[0])
server_id = server_id[0].split('server=')[1]
#start_timestamp is in Unix seconds, you can convert it to human readable time
start_timestamp, end_timestamp, server_id

('1673862903000', '1673863535000', '1681269718-129030')

In [7]:
#connecting to telegraf
telegraf_auth = json.load(open('telegraf_credentials'))
telegraf_client = InfluxDBClient(
    host="qtinfluxdb01.intra.qt.io",
    username=telegraf_auth['username'],
    password=telegraf_auth['password'],
    port=8086,
    ssl=True,
    database="coin_vms")

In [8]:
#all vm mesurements we keep
q = "show measurements"
tdf = pd.DataFrame(telegraf_client.query(q).get_points())
tdf

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,name
0,cpu
1,disk
2,diskio
3,ioping
4,kernel
5,kernel_vmstat
6,mem
7,netstat
8,processes
9,procstat_lookup


In [9]:
#example query
q = "SELECT *  FROM mem WHERE host =~ /{}(-|$)/ AND time >= {}ms and time <= {}ms".format(server_id,start_timestamp, end_timestamp)
tdf = pd.DataFrame(telegraf_client.query(q).get_points())
tdf.head()

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,time,active,available,available_percent,buffered,cached,commit_limit,committed_as,dirty,free,high_free,high_total,host,huge_page_size,huge_pages_free,huge_pages_total,inactive,low_free,low_total,mapped,page_tables,shared,slab,swap_cached,swap_free,swap_total,total,used,used_percent,vmalloc_chunk,vmalloc_total,vmalloc_used,wired,write_back,write_back_tmp
0,2023-01-16T09:55:20Z,0,7898918912,81.742812284351572,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,1764216832,18.257187715648424,0,0,0,0,0,0
1,2023-01-16T09:55:30Z,0,7518089216,77.801755198027777,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,2145046528,22.198244801972223,0,0,0,0,0,0
2,2023-01-16T09:55:40Z,0,7617515520,78.830679003240135,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,2045620224,21.169320996759868,0,0,0,0,0,0
3,2023-01-16T09:55:50Z,0,7438950400,76.982778645316728,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,2224185344,23.017221354683269,0,0,0,0,0,0
4,2023-01-16T09:56:00Z,0,7381303296,76.386211386745472,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,2281832448,23.613788613254524,0,0,0,0,0,0


In [10]:
#how many datapoints there are for give time slice (start_timestamp, end_timestamp))
tdf.shape

(62, 35)

In [11]:
q = "SELECT *  FROM cpu WHERE host =~ /{}(-|$)/ AND time >= {}ms and time <= {}ms".format(server_id,start_timestamp, end_timestamp)

tdf = pd.DataFrame(telegraf_client.query(q).get_points())
tdf.head()

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,time,cpu,host,usage_guest,usage_guest_nice,usage_idle,usage_iowait,usage_irq,usage_nice,usage_softirq,usage_steal,usage_system,usage_user
0,2023-01-16T09:55:30Z,cpu-total,1681269718-129030-Windows_10_22H2,0.0,0.0,33.520900321543408,0.0,0.000000000000000,0.0,0.0,0.0,21.704180064308680,44.774919614147912
1,2023-01-16T09:55:30Z,cpu0,1681269718-129030-Windows_10_22H2,0.0,0.0,31.093750000000000,0.0,0.000000000000000,0.0,0.0,0.0,27.187500000000000,41.718750000000000
2,2023-01-16T09:55:30Z,cpu1,1681269718-129030-Windows_10_22H2,0.0,0.0,33.283582089552240,0.0,4.477611940298507,0.0,0.0,0.0,16.567164179104477,45.671641791044777
3,2023-01-16T09:55:40Z,cpu-total,1681269718-129030-Windows_10_22H2,0.0,0.0,45.631825273010918,0.0,0.000000000000000,0.0,0.0,0.0,35.257410296411855,19.110764430577223
4,2023-01-16T09:55:40Z,cpu0,1681269718-129030-Windows_10_22H2,0.0,0.0,45.638629283489095,0.0,0.155763239875389,0.0,0.0,0.0,38.317757009345797,15.887850467289720


In [12]:
#Tony Sarajavi queries copied from his graana
q = "SELECT mean(total) as total, mean(buffered) as buffered, mean(cached) as cached, mean(total)-mean(free) as non_free, mean(used) as used, mean(total)-mean(available) as non_available  FROM mem WHERE host =~ /1681269718-122700(-|$)/ AND time >= 1673829274000ms and time <= 1673830968000ms GROUP BY time(30s), host ORDER BY asc"
tdf = pd.DataFrame(telegraf_client.query(q).get_points())
tdf.head()

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,time,total,buffered,cached,non_free,used,non_available
0,2023-01-16T00:34:30Z,16105586688.0,0.0,0.0,16105586688.0,1.795305472000000e+09,1.795305472000000e+09
1,2023-01-16T00:35:00Z,16105586688.0,0.0,0.0,16105586688.0,2.040752810666667e+09,2.040752810666666e+09
2,2023-01-16T00:35:30Z,16105586688.0,0.0,0.0,16105586688.0,2.493913770666667e+09,2.493913770666666e+09
3,2023-01-16T00:36:00Z,16105586688.0,0.0,0.0,16105586688.0,2.725780138666667e+09,2.725780138666666e+09
4,2023-01-16T00:36:30Z,16105586688.0,0.0,0.0,16105586688.0,2.993198421333333e+09,2.993198421333334e+09
